# Обобщения

Одна из сильных сторон C# - мощные типобезопасные обобщения, позволяющие использовать в качестве аргументов типа как ссылочные, так и значимые типы.

- Мотивация
- Внутреннее устройство
- Наследование
- Ковариантность и контравариантность типов
- Ограничения

# Мотивация

Очень часто хочется противоречащие вещи:

1. Один раз написать некоторый алгоритм/структуру данных так, чтобы она работала с разными типами данных.
2. Сохранить безопасность типов, чтобы избавиться от гемороя с обработкой ошибок типов.

Например, структура данных "Динамический массив" (список) по идее не зависит от типа содержимого. Мы не хотим переписывать этот код для каждого типа, с которым мы используем список.

Можно конечно сделать список object'ов (привет дженерикам из джавы), но так мы не сохраняем безопасность типов.

In [ ]:
public class MyList
{
    private const int DefaultCapacity = 4;

    private int[] data;
    private int capacity;
    
    public int Count { get; private set; }
    
    public int this[int index]
    {
        get
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            return data[index];
        }
        set
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            data[index] = value;
        }
    }
    
    public MyList() : this(DefaultCapacity) { }
    
    public MyList(int capacity)
    {
        if(capacity <= 0) throw new ArgumentException("Capacity must be positive");
        
        this.capacity = capacity;
        data = new int[capacity];
        Count = 0;
    }
    
    public void Add(T item)
    {
        if (Count == capacity) {
            Array.Resize(ref data, capacity *= 2);
        }
        data[Count] = item;
        ++Count;
    }
}

А вот и обобщённый вариант:

In [ ]:
public class MyList<T>
{
    private const int DefaultCapacity = 4;

    private T[] data;
    private int capacity;
    
    public int Count { get; private set; }
    
    public T this[int index]
    {
        get
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            return data[index];
        }
        set
        {
            if(index < 0 || index >= Count) throw new IndexOutOfRangeException($"Got index {index}, but size is {Count}.");
            data[index] = value;
        }
    }
    
    public MyList() : this(DefaultCapacity) { }
    
    public MyList(int capacity)
    {
        if(capacity <= 0) throw new ArgumentException("Capacity must be positive");
        
        this.capacity = capacity;
        data = new T[capacity];
        Count = 0;
    }
    
    public void Add(T item)
    {
        if (Count == capacity) {
            Array.Resize(ref data, capacity *= 2);
        }
        data[Count] = item;
        ++Count;
    }
}

`T` называется **параметром типа**, подставленное в него значение (напр. `int` в `List<int>`) - **аргументом типа**.

Также вводятся понятия **открытого** и **закрытого** типов.

**Закрытый тип** - обобщённый тип, в котором определены **все** параметры типа.

**Открытый тип** - обобщённый тип, в котором определены **не все** параметры типа.

In [20]:
// Закрытый тип
typeof(Dictionary<long, int>)

System.Collections.Generic.Dictionary<System.Int64,System.Int32>

In [21]:
// Открытый тип
typeof(Dictionary<,>)

System.Collections.Generic.Dictionary<TKey,TValue>

Обобщёнными могут быть: 

- Типы
 - Классы
 - Структуры
 - Интерфейсы
 - Делегаты
- Методы

Короче всё, что можно себе представить.

## Внутреннее устройство

Для каждого нового аргумента типа CLR генерирует **новый отдельный класс**, подставляя в него те аргументы, которые вы передали.

In [5]:
typeof(List<int>).ToString()

System.Collections.Generic.List`1[System.Int32]

In [10]:
typeof(List<double>).ToString()

System.Collections.Generic.List`1[System.Double]

In [4]:
typeof(Dictionary<long, string>).ToString()

System.Collections.Generic.Dictionary`2[System.Int64,System.Int32]

Благодаря этому достигается одно из преимуществ обобщений - **эффективная работа со значимыми типами (структурами)**. Для структур генерируется немного другой код, что позволяет избежать операций упаковки и распаковки.

Ещё одно важное вытекающее: **у каждого конкретного обобщённого типа своё статическое состояние**. 

In [12]:
class MyGeneric<T>
{
    public static int Value { get; set; }
}

In [13]:
MyGeneric<int>.Value = 4;
MyGeneric<double>.Value = 8;

In [14]:
MyGeneric<int>.Value

4

In [15]:
MyGeneric<double>.Value

8

## Наследование

Можно использовать параметры типа при наследовании и реализации интерфейса. 

Можно и не использовать.

In [24]:
public class MyGeneric<T1, T2> : List<T1>
{
    // ...
}

In [28]:
public class MyGeneric<T1, T2> : IComparable<int>
{
    public int CompareTo(int other) => 1;
}

## Ковариантность и контравариантность

Проблема:

string это object. Тогда коллекция string это коллекция object'ов - или нет?

In [40]:
List<string> myStrings = new List<string>() { "Hello", "World!" };
List<object> myObjects = myStrings;

Unhandled Exception: (2,26): error CS0029: Не удается неявно преобразовать тип "System.Collections.Generic.List<string>" в "System.Collections.Generic.List<object>".

А вот так оно почему-то работает. Магия?

In [42]:
IEnumerable<string> myStrings = new List<string>() { "Hello", "World!" };
IEnumerable<object> myObjects = myStrings;

foreach(var obj in myObjects)
{
    Console.WriteLine(obj);
}

Hello
World!


Было string -> object.

А object -> string?

In [45]:
public class ObjectConsumer : IComparable<object>
{
    public int CompareTo(object other) => 1;
}

IComparable<object> objComparable = new ObjectConsumer();
IComparable<string> strComparable = objComparable;

strComparable.CompareTo("Hello, world!")

1

Немного теории

> Параметры обобщённого типа могут быть:
> - **Инвариантными.** Параметр-тип не может изменяться. По умолчанию так.
> - **Ковариантными.** Аргумент-тип может быть преобразован от класса к одному из его базовых классов. В языке С# ковариантный тип обозначается ключевым словом out. Ковариантный параметр обобщенного типа может появляться только в выходной позиции, например, в качестве возвращаемого значения метода.
> - **Контравариантными.** Параметр-тип может быть преобразован от класса к классу, производному от него. В языке C# контравариантный тип обозначается ключевым словом in. Контравариантный параметр-тип может появляться только во входной позиции, например, в качестве аргументов метода.

In [33]:
// Делаем тип ковариативным
public interface IUseless<out T>
{
    T DoNothing(object obj);
}

In [31]:
// Тут сделать ковариативным нельзя, т.к. принимаем на вход.
public interface IUseless<out T>
{
    T DoNothing(T obj);
}

Unhandled Exception: (3,17): error CS1961: Недопустимое отклонение: Параметр типа "T" должен быть контравариантно, допустимым на "IUseless<T>.DoNothing(T)". "T" является ковариантный.

In [34]:
// Делаем тип контравариативным
public interface IUseless<in T>
{
    object DoNothing(T obj);
}

In [30]:
// Тут сделать контравариативным нельзя, т.к. возвращаем из метода.
public interface IUseless<in T>
{
    T DoNothing(T obj);
}

Unhandled Exception: (3,5): error CS1961: Недопустимое отклонение: Параметр типа "T" должен быть ковариантно, допустимым на "IUseless<T>.DoNothing(T)". "T" является контравариантный.

**Ко(нтра)вариация не работает для значимых типов!**

In [51]:
IEnumerable<string> myStrings = new List<string>() { "Hello", "World!" };
IEnumerable<object> myObjects = myStrings;

In [54]:
IEnumerable<int> myInts = new List<int>() { 1, 2, 3 };
// немогу...
IEnumerable<object> myObjects = myInts;

Unhandled Exception: (3,33): error CS0266: Не удается неявно преобразовать тип "System.Collections.Generic.IEnumerable<int>" в "System.Collections.Generic.IEnumerable<object>". Существует явное преобразование (возможно, пропущено приведение типов).

**Ковариация и контравариация расширяют возможности обобщений, но могут быть трудны в понимании.**

## Ограничения аргументов типов (constraints)

Дженерики - довольно жёсткая конструкция, так как одна из первостепенных задач - сохранить безопасность типов.

По умолчанию для объектов неопределённого типа доступны только методы класса object.

In [57]:
public T Min<T>(T left, T right)
{
    if(left.CompareTo(right) < 0)
        return left;
    return right;
}

Unhandled Exception: (3,13): error CS7036: Отсутствует аргумент, соответствующий требуемому формальному параметру "comparisonType" из "MemoryExtensions.CompareTo(ReadOnlySpan<char>, ReadOnlySpan<char>, StringComparison)".

Можно наложить на параметр-тип некоторые обязательства, а взамен - получить возможность пользоваться какими-то функциями. Например, можете обязать аргумент-тип реализовывать некоторый интерфейс. 

In [58]:
// Любой тип T должен быть IComparable<T>
public T Min<T>(T left, T right) where T : IComparable<T>
{
    if(left.CompareTo(right) < 0)
        return left;
    return right;
}

In [60]:
Min<int>(1, 5)

1

In [62]:
// Вывод типов компилятором - можно не указывать тип явно.
// Фича работает только с методами (по понятным причинам).
Min("bruh", "meh")

bruh

Можно довольно интересно использовать ограничения, связывая несколько параметров типов.

In [65]:
public List<TBase> ConvertIList<T, TBase>(IList<T> list) where T : TBase
{
    List<TBase> baseList = new List<TBase>(list.Count);
    for (Int32 index = 0; index < list.Count; index++) {
        baseList.Add(list[index]);
    }
    return baseList;
}

In [68]:
IList<String> ls = new List<String>() { "A String" };

IList<Object> lo = ConvertIList<String, Object>(ls);

IList<IComparable> lc = ConvertIList<String, IComparable>(ls);

IList<IComparable<String>> lcs = ConvertIList<String, IComparable<String>>(ls);

IList<String> ls2 = ConvertIList<String, String>(ls);

Можно применить следующие ограничения на параметр тип:
- Является классом: `where T : class`
- Является значимым типом: `where T : struct`
- Является типом (или производным от него): `where T : MyClass`
- Реализует интерфейс: `where T : IInterface`
- Имеет пустой конструктор: `where T : new()`

Порядок ниже

<img src="image3.png" style="width: 500px;">

Ограничения на енамы нет, если это нужно (зачем??), можно использовать статический конструктор класса, например.

In [ ]:
class GenericTypeThatRequiresEnum<TEnum>
{
    static GenericTypeThatRequiresEnum()
    {
        if(!typeof(TEnum).IsEnum)
        {
            throw new ArgumentException("TEnum must be an enumerated type!");
        }
    }
}

Интересный факт: ограничение where T : struct пропускает енамы

In [71]:
enum MyEnum 
{
    First, Second
}

public void MyGenericMethod<T>(T obj) where T : struct
{
    Console.WriteLine(typeof(T));
    Console.WriteLine(obj);
}

MyGenericMethod(MyEnum.First);

Submission#72+MyEnum
First


## Что нужно запомнить

- Для каждого конкретного набора аргументов типов генерируется отдельный класс/структура/интерфейс/делегат/метод.
- Сильная сторона - эффективная работа со значимыми (struct) аргументами типа.
- Ковариантность (out) - когда "можем засунуть  *производный* от заданного параметра. Допустим только тогда, когда тип используется лишь в возвращаемых значениях методов. Логика: "если мы возвращали Base, "
- Контравариантность (in) - когда можем использовать тип *базовый* для заданного параметра. Допустимо только тогда, когда тип используется лишь в параметрах методов. Логика: "если мы ожидаем параметр Base, от того, что нам начнут присылать Derived, нарушений не возникнет".
- С помощью ограничений на типы можно определить, какой набор свойств должен иметь аргумент тип. Благодаря этому можно безопасно использовать эти свойства в коде обобщенного класса/метода/чего-либо.